In [99]:
import pandas as pd
import requests
import pyodbc
from sqlalchemy import create_engine



In [100]:
json_url = 'https://jsonplaceholder.typicode.com/users'
response = requests.get(json_url)
if response.status_code == 200:
    json_data=response.json()
else:
    print(f"Request failed with status code {response.status_code}")

In [98]:
parsed_data = []

for item in json_data:
    parsed_item = {
        "customer_id": item.get("id"),
        "name": item.get("name"),
        "username": item.get("username"),
        "city": item.get("address").get("city"),
        "lat": item.get("address").get("geo").get("lat"),
        "lng": item.get("address").get("geo").get("lng"),
    }
    parsed_data.append(parsed_item)
    
users_df = pd.DataFrame(parsed_data)

In [4]:
#users dataframe
# users_df

In [5]:
orders_details=pd.read_csv('AiQ.csv')

In [6]:
merged_data = pd.merge(orders_details, users_df, on='customer_id', how='left')

In [66]:
merged_data['lng']=merged_data['lng'].astype(float)
merged_data['lat']=merged_data['lat'].astype(float)

In [49]:
import pandas as pd

# Create a dictionary with locations and addresses
locations_dict = {
    "city": ["Roscoeview", "Aliyaview", "Bartholomebury", "McKenziehaven", "Lebsackbury",
                 "Howemouth", "South Elvis", "Gwenborough", "Wisokyburgh", "South Christy"],
    "Address": ["Roscoe Village, Chicago, IL, USA",
                "303, Al-Falah Building, Shahrah-e-Quaid-e-Azam, Garhi Shahu, Lahore, Punjab 54000, Pakistan",
                "Null",  # Address not found
                "11 Hillcross St # 83, McKenzie, AL 36456, USA",
                "Null",  # Address not found
                "Null",  # Address not found
                "Memphis, TN 38106, USA",
                "Guisborough TS14, UK",
                "Winchburgh, UK",
                "2490 S Woodworth Loop Suite 301, Palmer, AK 99645, USA"]
}

df = pd.DataFrame(locations_dict)
location_fictious_address = pd.merge(df, merged_data, on='city', how='left')
location_fictious_address = location_fictious_address[['city', 'Address', 'lat', 'lng']]
location_fictious_address = location_fictious_address.drop_duplicates()
location_fictious_address

,city,Address,lat,lng
0,Roscoeview,"Roscoe Village, Chicago, IL, USA",-31.8129,62.5342
102,Aliyaview,"303, Al-Falah Building, Shahrah-e-Quaid-e-Azam...",-14.3990,-120.7677
208,Bartholomebury,Null,24.6463,-168.8889
314,McKenziehaven,"11 Hillcross St # 83, McKenzie, AL 36456, USA",-68.6102,-47.0653
416,Lebsackbury,Null,-38.2386,57.2232
527,Howemouth,Null,24.8918,21.8984
619,South Elvis,"Memphis, TN 38106, USA",29.4572,-164.2990
716,Gwenborough,"Guisborough TS14, UK",-37.3159,81.1496
795,Wisokyburgh,"Winchburgh, UK",-43.9509,-34.4618
904,South Christy,"2490 S Woodworth Loop Suite 301, Palmer, AK 99...",-71.4197,71.7478


In [54]:
api_key = 'd7cd1f0eb6f54b95ef100b2d83d6b65c'
weather_data_list = []
for index, row in location_fictious_address.iterrows():
    lat = row['lat']
    lon = row['lng']
    address = row['Address']
    url = f"https://api.openweathermap.org/data/2.5/weather?q={address}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        weather_data = response.json()
        weather_data_list.append(weather_data)
    else:
        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            weather_data = response.json()
            weather_data_list.append(weather_data)


In [89]:

parsed_data=[]
for items in weather_data_list:
    temp_celsius = items.get('main').get('temp') - 273.15
    feels_like_celsius = items.get('main').get('feels_like') - 273.15
    parsed_items = {
        "lon": items.get('coord').get('lon'),
        "lat": items.get('coord').get('lat'),
        "weather_main": items.get('weather')[0].get('main'),
        "weather_description": items.get('weather')[0].get('description'),
        "temp": items.get('main').get('temp'),
        "feels_like": items.get('main').get('feels_like'),
        "pressure": items.get('main').get('pressure'),
        "humidity": items.get('main').get('humidity'),
        "wind_speed": items.get('wind').get('speed'),
        "wind_deg": items.get('wind').get('deg'),
        "clouds": items.get('clouds').get('all'),  
        "temp": temp_celsius,
        "feels_like": feels_like_celsius,
    }
    parsed_data.append(parsed_items)

weather_df = pd.DataFrame(parsed_data)


In [95]:
weather_df

,lon,lat,city,weather_main,weather_description,temp
0,62.5342,-31.8129,Roscoeview,Clouds,broken clouds,19.78
1,-120.7677,-14.3990,Aliyaview,Rain,light rain,25.92
2,-168.8889,24.6463,Bartholomebury,Clouds,overcast clouds,20.16
3,-47.0653,-68.6102,McKenziehaven,Clouds,overcast clouds,-1.15
4,57.2232,-38.2386,Lebsackbury,Clouds,overcast clouds,16.05
5,21.8984,24.8918,Howemouth,Clear,clear sky,17.65
6,-90.0516,35.1490,NaN,Clear,clear sky,-3.53
7,81.1496,-37.3159,Gwenborough,Clouds,few clouds,14.17
8,-34.4618,-43.9509,Wisokyburgh,Clouds,scattered clouds,11.61
9,71.7478,-71.4197,South Christy,Clear,clear sky,-14.75


In [90]:
# weather_df['lon']=weather_df['lon'].astype(float)
# weather_df['lat']=weather_df['lat'].astype(float)
# users_df['lng']=users_df['lng'].astype(float)
# users_df['lat']=users_df['lat'].astype(float)
weather_df=pd.merge(weather_df,users_df,left_on=['lon','lat'],right_on=['lng','lat'],how='left')
# weather_df=weather_df[['lon','lat','city','weather_main','weather_description','temp']]


In [92]:
weather_df=weather_df[['lon','lat','city','weather_main','weather_description','temp']]

In [93]:
weather_df

,lon,lat,city,weather_main,weather_description,temp
0,62.5342,-31.8129,Roscoeview,Clouds,broken clouds,19.78
1,-120.7677,-14.3990,Aliyaview,Rain,light rain,25.92
2,-168.8889,24.6463,Bartholomebury,Clouds,overcast clouds,20.16
3,-47.0653,-68.6102,McKenziehaven,Clouds,overcast clouds,-1.15
4,57.2232,-38.2386,Lebsackbury,Clouds,overcast clouds,16.05
5,21.8984,24.8918,Howemouth,Clear,clear sky,17.65
6,-90.0516,35.1490,NaN,Clear,clear sky,-3.53
7,81.1496,-37.3159,Gwenborough,Clouds,few clouds,14.17
8,-34.4618,-43.9509,Wisokyburgh,Clouds,scattered clouds,11.61
9,71.7478,-71.4197,South Christy,Clear,clear sky,-14.75


In [14]:
total_sales_dataset = pd.merge(merged_data, weather_df, left_on=['lng','lat'], right_on=['lon','lat'], how='left')


In [15]:
total_sales_dataset = total_sales_dataset.drop(columns=['lon'])

In [16]:
total_sales_dataset

,order_id,customer_id,product_id,quantity,price,order_date,name,username,city,lat,lng,weather_main,weather_description,temp,feels_like,pressure,humidity,wind_speed,wind_deg,clouds
0,2334,5,40,3,35.60,2022-06-21,Chelsey Dietrich,Kamren,Roscoeview,-31.8129,62.5342,Clouds,scattered clouds,20.31,20.25,1019,71,5.72,159,45
1,6228,8,13,7,36.52,2023-03-08,Nicholas Runolfsdottir V,Maxime_Nienow,Aliyaview,-14.3990,-120.7677,Clouds,few clouds,26.41,26.41,1014,71,6.52,94,18
2,7784,9,44,4,46.56,2023-04-22,Glenna Reichert,Delphine,Bartholomebury,24.6463,-168.8889,Clouds,broken clouds,20.17,19.81,1025,60,9.50,70,61
3,6588,5,26,1,15.87,2022-10-23,Chelsey Dietrich,Kamren,Roscoeview,-31.8129,62.5342,Clouds,scattered clouds,20.31,20.25,1019,71,5.72,159,45
4,5910,8,32,10,77.00,2022-10-05,Nicholas Runolfsdottir V,Maxime_Nienow,Aliyaview,-14.3990,-120.7677,Clouds,few clouds,26.41,26.41,1014,71,6.52,94,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7967,6,34,10,14.44,2022-07-26,Mrs. Dennis Schulist,Leopoldo_Corkery,South Christy,-71.4197,71.7478,Clear,clear sky,-20.28,-27.28,990,71,3.80,98,1
996,8308,6,16,1,61.38,2023-03-04,Mrs. Dennis Schulist,Leopoldo_Corkery,South Christy,-71.4197,71.7478,Clear,clear sky,-20.28,-27.28,990,71,3.80,98,1
997,6938,4,18,7,62.96,2022-11-27,Patricia Lebsack,Karianne,South Elvis,29.4572,-164.2990,Clouds,overcast clouds,15.58,14.76,1028,60,6.84,13,100
998,5921,9,37,9,52.89,2022-09-22,Glenna Reichert,Delphine,Bartholomebury,24.6463,-168.8889,Clouds,broken clouds,20.17,19.81,1025,60,9.50,70,61


In [60]:
# Replace the placeholders with your actual SQL Server connection details
def push_df_to_db(df,table_name):
    host = 'DESKTOP-6P1LUS8'
    database_name = 'AiQ'
    driver = 'SQL Server Native Client 11.0'
    connection_string = f"mssql+pyodbc://{host}/{database_name}?driver={driver}"
    engine = create_engine(connection_string)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    engine.dispose()


In [94]:
push_df_to_db(total_sales_dataset,'total_sales_dataset')
push_df_to_db(users_df,'users_details')
push_df_to_db(location_fictious_address,'location_fictious_address')
push_df_to_db(weather_df,'weather_details')
push_df_to_db(orders_details,'orders_details')